In [61]:
import os

# Get the directory of the current script
base_dir = os.getcwd()
try:
    if folder:
        base_dir = os.path.join(base_dir, folder)
except:
    pass

In [62]:
from rdflib import *
import os

In [63]:
def get_graph():
    
    g = Graph()
    g.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
    g.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
    g.bind('odrl', 'http://www.w3.org/ns/odrl/2/')
    
    return g

In [64]:
contract = Graph()
contract.parse(os.path.join(base_dir, '../../../FederatedComputationalGovernance/SemanticDataModel/sdm.ttl'), format="ttl")

<Graph identifier=Nde3566f1203148f8a8b5b5c4274763e3 (<class 'rdflib.graph.Graph'>)>

In [65]:
# Define the URI for the specific data product
dp = URIRef("http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#UPENN-GBM_clinical_info_v2")
pc = URIRef("http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#pc4")


select_query = f"""
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#>
PREFIX ab: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

CONSTRUCT {{
     ab:OP a tb:Operation ;
               tb:hasInput ab:data ;
               tb:hasInput ?attribute ;
               tb:hasAbstract odrl:Constraint ;
               tb:hasOutput ab:boolean .
     ab:test tb:nextStep ab:OP.
     
     ab:OP2 a tb:Operation ;
               tb:hasInput ab:data ;
               tb:hasInput ?attribute ;
               tb:hasAbstract odrl:Constraint ;
               tb:hasOutput ab:boolean .
     ab:test tb:nextStep ab:OP.
     
}}
WHERE {{
    ab:UPENN-GBM_clinical_info_v21csv tb:hasDC ?dc .
    ?dc tb:hasPolicy ?policy .
    FILTER (?policy = ab:p4)
    ?policy a odrl:Policy ;
            odrl:prohibition ?prohibition .
    ?prohibition odrl:target ?target .
    
    ?prohibition odrl:constraint ?logicalConstraint .
    ?logicalConstraint a odrl:LogicalConstraint ;
                       odrl:operator odrl:and ;
                       odrl:andConstraints ?constraint1, ?constraint2 .
                    
    ?constraint2 odrl:leftOperand ?leftOperand2 ;
                    odrl:operator ?operator2 .


    BIND(?target AS ?attribute)
    BIND(IRI(CONCAT(STR(ab:), STRUUID())) AS ?op_uri_1)
    BIND(IRI(CONCAT(STR(ab:), STRUUID())) AS ?op_uri_2)
}}

"""

# Execute the SPARQL SELECT query
results = contract.query(select_query)


# Create a new graph for the results
result_graph = get_graph()

# Add the constructed triples to the result graph
for triple in results:
    result_graph.add(triple)

# Print the new triples
for triple in result_graph:
    print(triple)
    
    
    
# Optionally, serialize the modified graph to a file
result_graph.serialize("test.ttl", format="turtle")

<Graph identifier=Nbc2afed5c1604d3f826217b6925a72c3 (<class 'rdflib.graph.Graph'>)>